# Liquid Instrument Data Logger Examples

To demonstrates use of the Datalogger instrument to log time-series voltage data or stream.

## Setup the Instrument

Include the library and create the instrument object

In [36]:
import os
import time

from moku.instruments import Datalogger

### Connect the instrument
Connect to your Moku by its ip address ('192.168.###.###' or ipv6 addr) using Datalogger

In [38]:
i = Datalogger('[fe80::7269:79ff:feb0:a1c%10]', force_connect=True)

### Configure the instrument

In [ ]:
# Set the frontend
i.set_frontend(channel=1, impedance='1MOhm', coupling="DC", range="40Vpp")
i.set_frontend(channel=2, impedance='1MOhm', coupling="DC", range="40Vpp")
i.set_frontend(channel=3, impedance='1MOhm', coupling="DC", range="40Vpp")
i.set_frontend(channel=4, impedance='1MOhm', coupling="DC", range="40Vpp")

In [ ]:
# Set the sample rate from 10 to 10e6 Hz
i.set_samplerate(1000) 

In [ ]:
# Set the acquisition mode (Normal, Precision, PeakDetect)
i.set_acquisition_mode(mode='Precision')

In [ ]:
# Summarize the settings of current instrument
print(i.summarize())

## Aquire Data


### Acquire data and log to a file

In [ ]:
# Stop an existing log, if any, then start a new one. 10 seconds of both channels.
logFile = i.start_logging(duration=10)

# Track progress percentage of the data logging session
is_logging = True
while is_logging:
    # Wait for the logging session to progress by sleeping 0.5sec
    time.sleep(0.5)
    # Get current progress percentage and print it out
    progress = i.logging_progress()
    remaining_time = int(progress['time_to_end'])
    is_logging = remaining_time > 1
    print(f"Remaining time {remaining_time} seconds")
    
# Download log from Moku, use liconverter to convert this .li file to .csv
i.download("persist", logFile['file_name'], os.path.join(os.getcwd(), logFile['file_name']))
print("Downloaded log file to local directory.")

### Acquire data stream and plot

In [ ]:
import matplotlib.pyplot as plt

# Set the sample rate 
i.set_samplerate(10e3)

# stream the data for 10 seconds..
i.start_streaming(10)

# Set up the plotting parameters
plt.ion()
fig, ax = plt.subplots()
plt.grid(visible=True)
plt.ylim([-1, 1])
line1, = plt.plot([])

# This loops continuously updates the plot with new data
while True:
    # get the chunk of streamed data
    data = i.get_stream_data()
    if data:
        plt.xlim([data['time'][0], data['time'][-1]])
        # Update the plot
        line1.set_ydata(data['ch1'])
        line1.set_xdata(data['time'])
        fig.canvas.draw()
        plt.pause(0.001) # Add a short pause to improve animation smoothness

## Close the Instrument

In [39]:
i.relinquish_ownership()